In [5]:
%matplotlib inline
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
from matplotlib import pylab
from six.moves import range
import pickle
import random

In [6]:
# ====================
#  TOY DATA GENERATOR
# ====================
class ToySequenceData(object):
    """ Generate sequence of data with dynamic length.
    This class generate samples for training:
    - Class 0: linear sequences (i.e. [0, 1, 2, 3,...])
    - Class 1: random sequences (i.e. [1, 3, 10, 7,...])
    NOTICE:
    We have to pad each sequence to reach 'max_seq_len' for TensorFlow
    consistency (we cannot feed a numpy array with unconsistent
    dimensions). The dynamic calculation will then be perform thanks to
    'seqlen' attribute that records every actual sequence length.
    """
    def __init__(self, n_samples=1000, max_seq_len=20, min_seq_len=3,
                 max_value=1000):
        self.data = []
        self.labels = []
        self.seqlen = []
        for i in range(n_samples):
            # Random sequence length
            len = random.randint(min_seq_len, max_seq_len)
            # Monitor sequence length for TensorFlow dynamic calculation
            self.seqlen.append(len)
            # Add a random or linear int sequence (50% prob)
            if random.random() < .5:
                # Generate a linear sequence
                rand_start = random.randint(0, max_value - len)
                s = [[float(i)/max_value] for i in
                     range(rand_start, rand_start + len)]
                # Pad sequence for dimension consistency
                s += [[0.] for i in range(max_seq_len - len)]
                self.data.append(s)
                self.labels.append([1., 0.])
            else:
                # Generate a random sequence
                s = [[float(random.randint(0, max_value))/max_value]
                     for i in range(len)]
                # Pad sequence for dimension consistency
                s += [[0.] for i in range(max_seq_len - len)]
                self.data.append(s)
                self.labels.append([0., 1.])
        self.batch_id = 0

    def next(self, batch_size):
        """ Return a batch of data. When dataset end is reached, start over.
        """
        if self.batch_id == len(self.data):
            self.batch_id = 0
        batch_data = (self.data[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_labels = (self.labels[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_seqlen = (self.seqlen[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        self.batch_id = min(self.batch_id + batch_size, len(self.data))
        return batch_data, batch_labels, batch_seqlen

In [59]:
lstm_size = 64
max_length = 50
batch_size = 10

trainset = ToySequenceData(n_samples=100000, max_seq_len=max_length)
print(trainset.data[0])
print(trainset.labels[0])
print(trainset.seqlen[0])
print(trainset.data[1])
print(trainset.labels[1])
print(trainset.seqlen[1])

[[0.096], [0.097], [0.098], [0.099], [0.1], [0.101], [0.102], [0.103], [0.104], [0.105], [0.106], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
[1.0, 0.0]
11
[[0.767], [0.768], [0.769], [0.77], [0.771], [0.772], [0.773], [0.774], [0.775], [0.776], [0.777], [0.778], [0.779], [0.78], [0.781], [0.782], [0.783], [0.784], [0.785], [0.786], [0.787], [0.788], [0.789], [0.79], [0.791], [0.792], [0.793], [0.794], [0.795], [0.796], [0.797], [0.798], [0.799], [0.8], [0.801], [0.802], [0.803], [0.804], [0.805], [0.806], [0.807], [0.808], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
[1.0, 0.0]
42


In [48]:
graph = tf.Graph()

with graph.as_default():
  inputs = tf.placeholder(tf.float32, shape=[None, max_length, 1])  
  labels = tf.placeholder(tf.float32, shape=[None, 2])
  sequence_lengths = tf.placeholder(tf.int32, shape=(None,))
    
  cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_size, state_is_tuple=True)
  outputs, state = tf.nn.dynamic_rnn(cell,
                      inputs,
                      dtype=tf.float32,
                      sequence_length=sequence_lengths
                      )
    
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([lstm_size, 2], -0.1, 0.1), name='w')
  b = tf.Variable(tf.zeros([2]), name='b')
  logits = tf.nn.xw_plus_b(state.h, w, b)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))

  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  train_predictions = tf.nn.softmax(logits)

In [60]:
with tf.Session(graph=graph) as sess:
  sess.run(tf.initialize_all_variables())
  
  print("Run session...")
  for step in range(50001):
    batch_inputs, batch_labels, batch_seqlen = trainset.next(batch_size)
    feed_dict = { inputs: batch_inputs, labels: batch_labels, sequence_lengths: batch_seqlen }
    _, l, predictions = sess.run([optimizer, loss, train_predictions], feed_dict=feed_dict)

    if step % 1000 == 0:
      print('Step %d:' % step)
      print('Training set:')
      print('  Loss       : ', l)
      print('  Input      : ', batch_inputs[0])
      print('  Label      : ', batch_labels[0])
      print('  Prediction : ', predictions[0])
      print("="*50)

Run session...
Step 0:
Training set:
  Loss       :  0.674321
  Input      :  [[0.096], [0.097], [0.098], [0.099], [0.1], [0.101], [0.102], [0.103], [0.104], [0.105], [0.106], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Label      :  [1.0, 0.0]
  Prediction :  [ 0.50400728  0.49599278]
Step 1000:
Training set:
  Loss       :  0.679651
  Input      :  [[0.335], [0.86], [0.608], [0.115], [0.88], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
  Label      :  [0.0, 1.0]
  Prediction :  [ 0.53365505  

In [58]:
batch_inputs, batch_labels, batch_seqlen = trainset.next(batch_size)
print(batch_inputs[0])

[[0.091], [0.107], [0.435], [0.353], [0.914], [0.38], [0.359], [0.577], [0.627], [0.295], [0.236], [0.839], [0.738], [0.718], [0.923], [0.914], [0.005], [0.643], [0.945], [0.492], [0.182], [0.284], [0.794], [0.164], [0.25], [0.38], [0.434], [0.065], [0.398], [0.94], [0.179], [0.056], [0.16], [0.972], [0.96], [0.293], [0.297], [0.768], [0.473], [0.0], [0.686], [0.367], [0.05], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
